<a href="https://colab.research.google.com/github/Joongeun/Internship/blob/Topic-Modeling/Twitter_roBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install spacy
import spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import json

nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words

posts = []

with open('/content/sample_data/QuitVaping_submissions', 'r', encoding='utf-8') as f:
  while True:
    try:
      data = json.loads(f.readline())
      text = [word for word in data['selftext'].split() if not word in stop_words]
      posts.append(' '.join(text))
      #posts.append(data['selftext'])
    except:
      break

posts = list(set(posts))
# Some deleted/remoed posts have other content in it --> check if "[removed]" or "[deleted]" is in the post (e.x: deleted polls have links along with the word "[deleted]")
for i in posts:
  if "[removed]" in i or "[deleted]" in i or len(i) < 1:
    posts.remove(i)
print(len(posts))

7796


In [22]:
for i in posts:
  if len(i) < 30:
    print(i)

Dm buddy code
I’ve seen conflicting data
Headaches brutal
Does know mood swings better?
Think I’m looking motivation.
Wish luck. I need it.
It wasn’t worth hating rn
Do find vaping affects mood?
Onwards
Don’t thinking relapsing!
day 1 tomorrow
Title says it’s all.
Glad success stories, strong!
Dm want join
Just wanna guys luck like
In 8 hours I'll hit 1 week!
tips dealing fear quitting?
AMOR FATI
3 weeks vape free btw.
It got easier :)
Let’s shit
Read title.
Genuinely wondering.
Which I read?
Day 4, insatiable appetite.
I know bad I miss
I feel like garbage.
I postponing quit date.
The cravings relentless
I Ellis_447
Back work. That’s I got
Ugh.
How I stop buying more?
When cravings stop!?
FUCK
Erectile dysfunction.
How I begin process quitting
Help
It’s stress
Is worst gone??
I wanted vent lmao.
1 hour down, lifetime
think heavy life change .
Please
Convince vape.
Just gotta post. Idk. Nothing
How’s day going?
I constantly think shit brain
skin? health? energy?
Almost big one-zero
Can

In [5]:
# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']
encoded_posts = []
for post in posts:
  encoded_posts.append(tokenizer(post, return_tensors='pt'))
print(len(encoded_posts), encoded_posts[0])

7796 {'input_ids': tensor([[    0,  2847,    38,    17,    27,   548, 40634,   481,   360,     4,
            38,    17,    27,   119, 28342,     4,   440,   809,     4,   125,
           313, 14223,   313,    38,    17,    27,   119,  4441,   319,   328,
         11182,   102,     6, 29965,     6,   364,  3998, 18022, 13361,   152,
          2340, 17487,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [20]:
def get_sentiment_prob(encoded_post):
  try:
    output = model(**encoded_post)
  except:
    return True
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  score_dict = {}
  #Scores are printed as a probability - x% negative, y% neutral, z% positive
  for i in range(len(scores)):
      l = labels[i]
      s = scores[i]
      score_dict[l] = s
  return score_dict

scores = []
too_long_posts = []
for ind, i in enumerate(encoded_posts):
  ret_val = get_sentiment_prob(i)
  if ret_val == True:
    too_long_posts.append(posts[ind])
  else:
    scores.append(ret_val)
print(len(scores))
print(too_long_posts)

counter = {'Negative': 0, 'Neutral': 0, 'Positive': 0}
for i in scores:
  score = [i['Negative'], i['Neutral'], i['Positive']]
  counter[labels[score.index(max(score))]] += 1

x = lambda y: y/len(scores)

for i in labels:
  print(i, x(counter[i]))

7700
['Hey everyone, I unsub subreddit time I vaped Christmas Day. Prior I vaped day New Years Day 2015, prior I smoked pretty daily I don\'t know, 7-10 years. Before I un-sub though, I thought right thing drop cents worked hopes benefit it, way I\'ve benefitted encouragement people posted me. &amp;#x200B; So facts personal situation conclusions I drawn: 1 - I wanted stop vaping YEARS reason, I manage it. Conclusion: You WANT it. But WANTING isn\'t it\'s own. 2 - I started vaping 3mg, mg stayed time vape rigs got bigger bigger. Over 2 years though, mixing liquid I stepped nic level 3mg, 1.5mg, 0.75mg finally 0.325mg. I stayed 0.3mg level year little nicotine big challenge, I think titrating nec level helped. Conclusion: Weaning nic level gradually DOES help. 3 - I tested positive Omicron Christmas Day evening (Yep sucked). That day I vaped, 48 days ago today. Now first, think oh main thing, I\'ll tell now, I symptomatic 3 4 days testing NEGATIVE time I vaping like maniac. I didn\'t REM

In [21]:
x = lambda y: y/len(posts)

for i in labels:
  print(i, x(counter[i]))

Negative 0.4727272727272727
Neutral 0.3288311688311688
Positive 0.19844155844155845
